In [2]:
from yehuda_code.rna_access import RNAAccess
import pandas as pd
import numpy as np
import pickle
from asodesigner.util import get_antisense
from consts import *
from read_human_genome import get_locus_to_data_dict
from file_utils import read_human_genome_fasta_dict

ModuleNotFoundError: No module named 'file_util'

In [8]:
# conda activate aso_design
import pandas as pd
from scripts.data_genertion.consts import *
from asodesigner.file_utils import read_human_genome_fasta_dict
from asodesigner.consts import *
import numpy as np
from asodesigner.util import get_antisense
import pickle
from read_human_genome import get_locus_to_data_dict
from file_utils import read_human_genome_fasta_dict
all_data = pandas.read_csv(DATA_PATH / 'data_from_article_fixed.csv')

In [5]:
SEQUENCE = 'Sequence'
INHIBITION = 'Inhibition(%)'
CANONICAL_GENE = 'Canonical Gene Name'
CELL_LINE_ORGANISM = 'Cell line organism'
VOLUME = 'ASO_volume(nM)'
TREATMENT_PERIOD = 'Treatment_Period(hours)'
CELL_LINE = 'Cell_line'
TRANSFECTION = 'Transfection'
DENSITY = 'Density(cells/well)'
MODIFICATION = 'Modification'

In [9]:
all_data_no_nan = all_data.dropna(subset=[INHIBITION]).copy()
all_data_no_nan.loc[:, 'log_inhibition'] = -np.log(-all_data_no_nan[INHIBITION] + 100.001)

In [10]:
all_data_no_nan_human = all_data_no_nan[all_data_no_nan[CELL_LINE_ORGANISM] == 'human']
genes = all_data_no_nan[CANONICAL_GENE].copy()
genes_u = list(set(genes))
genes_u.remove('HBV')
genes_u.remove('negative_control')

In [11]:
# Load gene information from cache or generate it if needed
cache_path = CACHE_DIR / 'gene_to_data_simple_cache.pickle'
if not cache_path.exists():
    gene_to_data = get_locus_to_data_dict(include_introns=True, gene_subset=genes_u)
    with open(cache_path, 'wb') as f:
        pickle.dump(gene_to_data, f)
else:
    with open(cache_path, 'rb') as f:
        gene_to_data = pickle.load(f)


In [12]:
# Filter data to keep only rows with valid gene information
all_data_human_gene = all_data_no_nan_human[all_data_no_nan_human[CANONICAL_GENE].isin(genes_u)].copy()

# Define names for new columns
SENSE_SEQUENCE = 'sense_sequence'
PRE_MRNA_SEQUENCE = 'pre_mrna_sequence'
SENSE_START = 'sense_start'
SENSE_LENGTH = 'sense_length'

# Initialize new columns
all_data_human_gene[SENSE_SEQUENCE] = ""
all_data_human_gene[PRE_MRNA_SEQUENCE] = ""
all_data_human_gene[SENSE_START] = np.zeros_like(all_data_human_gene[CANONICAL_GENE], dtype=int)
all_data_human_gene[SENSE_LENGTH] = np.zeros_like(all_data_human_gene[CANONICAL_GENE], dtype=int)

# Iterate over each row and compute the antisense complement and the gene's pre-mRNA
for index, row in all_data_human_gene.iterrows():
    gene_name = row[CANONICAL_GENE]

    if gene_name not in gene_to_data:
        continue  # Skip genes not found in genome annotation

    locus_info = gene_to_data[gene_name]
    pre_mrna = locus_info.full_mrna
    antisense = row[SEQUENCE]
    sense = get_antisense(antisense)
    idx = pre_mrna.find(sense)

    # Store computed sequences in new columns
    all_data_human_gene.loc[index, SENSE_START] = idx
    all_data_human_gene.loc[index, SENSE_LENGTH] = len(antisense)
    all_data_human_gene.at[index, SENSE_SEQUENCE] = sense
    all_data_human_gene.at[index, PRE_MRNA_SEQUENCE] = pre_mrna


In [28]:
all_data_human_gene.head()

,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,Modification,...,Smiles,Inhibition(%),seq_length,Canonical Gene Name,Cell line organism,log_inhibition,sense_sequence,pre_mrna_sequence,sense_start,sense_length
0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,cEt/5-methylcytosines/deoxy,...,Cc1cn([C@H]2C[C@H](OP(=O)([S-])[O]C[C@H]3O[C@@...,33.0,16,KRAS,human,-4.204708,TAGCATTTGTTTTAGC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",41212,16
1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,cEt/5-methylcytosines/deoxy,...,Cc1cn([C@H]2C[C@H](OP(=O)([S-])[O]C[C@]34O[C@@...,7.0,16,KRAS,human,-4.532610,GATATTCACCATTATA,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",23686,16
2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,cEt/5-methylcytosines/deoxy,...,Cc1cn([C@H]2C[C@H](OP(=O)([S-])[O]C[C@H]3O[C@@...,62.0,16,KRAS,human,-3.637612,CCAGAAATCTTCATGC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",43363,16
3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,cEt/5-methylcytosines/deoxy,...,Cc1cn([C@H]2C[C@H](OP(=O)([S-])[O]C[C@H]3O[C@@...,28.0,16,KRAS,human,-4.276680,TTTGAAGATATTCACC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",23680,16
4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,cEt/5-methylcytosines/deoxy,...,Cc1cn([C@H]2C[C@H](OP(=O)([S-])[O]C[C@H]3O[C@@...,36.0,16,KRAS,human,-4.158899,CATACTAGTACAAGTG,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",41168,16


In [13]:
def get_sense_with_flanks(pre_mrna: str, sense_start: int, sense_length: int, flank_size: int) -> str:
    """
    Re  turns the sense sequence with `flank_size` nucleotides on each side (if available).
    If near the edge, it will not go out of bounds.

    Parameters:
    - pre_mrna: The full pre-mRNA sequence (5' -> 3')
    - sense_start: Start index of the sense sequence within pre_mrna
    - sense_length: Length of the sense sequence (usually same as antisense length)
    - flank_size: Number of nucleotides to include on each side (upstream and downstream)

    Returns:
    - str: The flanked sense sequence
    """
    # Ensure indices are within bounds
    start = max(0, sense_start - flank_size)
    end = min(len(pre_mrna), sense_start + sense_length + flank_size)

    return pre_mrna[start:end]


In [14]:
FLANK_SIZE = 120  # Change this as needed
FLANKED_SENSE_COL = f'sense_with_flank_{FLANK_SIZE}nt'

# Create new column with flanked sequences
all_data_human_gene[FLANKED_SENSE_COL] = all_data_human_gene.apply(
    lambda row: get_sense_with_flanks(
        row['pre_mrna_sequence'],
        row['sense_start'],
        row['sense_length'],
        flank_size=FLANK_SIZE
    ) if row['sense_start'] != -1 else "",  # Handle cases where sense was not found
    axis=1
)


In [15]:
all_data_human_gene.head()

,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,Modification,...,Inhibition(%),seq_length,Canonical Gene Name,Cell line organism,log_inhibition,sense_sequence,pre_mrna_sequence,sense_start,sense_length,sense_with_flank_120nt
0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,cEt/5-methylcytosines/deoxy,...,33.0,16,KRAS,human,-4.204708,TAGCATTTGTTTTAGC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",41212,16,"(A, T, G, G, T, A, A, A, A, A, G, A, A, G, A, ..."
1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,cEt/5-methylcytosines/deoxy,...,7.0,16,KRAS,human,-4.532610,GATATTCACCATTATA,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",23686,16,"(C, T, C, T, T, G, G, A, T, A, T, T, C, T, C, ..."
2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,cEt/5-methylcytosines/deoxy,...,62.0,16,KRAS,human,-3.637612,CCAGAAATCTTCATGC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",43363,16,"(T, A, G, A, C, A, G, A, A, C, C, C, T, A, T, ..."
3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,cEt/5-methylcytosines/deoxy,...,28.0,16,KRAS,human,-4.276680,TTTGAAGATATTCACC,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",23680,16,"(A, C, C, T, G, T, C, T, C, T, T, G, G, A, T, ..."
4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,cEt/5-methylcytosines/deoxy,...,36.0,16,KRAS,human,-4.158899,CATACTAGTACAAGTG,"(G, A, T, T, T, T, C, C, T, A, G, G, C, G, G, ...",41168,16,"(A, T, T, A, G, T, T, C, G, A, G, A, A, A, T, ..."


Running RNAaccess on the sense with flanks

In [18]:
from tqdm import tqdm  # for progress bar
from yehuda_code.access_calculator import AccessCalculator

# Define parameters
#access_size = len(seq)
min_gc = 0
max_gc = 100
gc_ranges = 1
access_win_size = 100
seed_size = 6
seed_sizes = [seed_size * m for m in range(1, 4)]

row = all_data_human_gene.iloc[0]  # take the first row
seq = row[f'sense_with_flank_{FLANK_SIZE}nt']
sense_start = row['sense_start']
sense_length = row['sense_length']

try:
    # Run accessibility calculation
    df_access = AccessCalculator.calc(
        seq, len(seq),
        min_gc, max_gc, gc_ranges,
        access_win_size, seed_sizes
    )

    # Calculate start and end of sense region within the flanked sequence
    flank_start = max(0, sense_start - FLANK_SIZE)
    sense_start_in_flank = sense_start - flank_start
    sense_end_in_flank = sense_start_in_flank + sense_length

    # Extract mean accessibility in the sense region
    if 0 <= sense_start_in_flank < len(df_access) and sense_end_in_flank <= len(df_access):
        mean_access = df_access['avg_access'].iloc[sense_start_in_flank:sense_end_in_flank].mean()
    else:
        mean_access = None

    print("Mean accessibility:", mean_access)

except Exception as e:
    print(f"Error on test row: {e}")


ModuleNotFoundError: No module named 'rna_access'

In [17]:
from tqdm import tqdm  # for progress bar
from yehuda_code.access_calculator import AccessCalculator

# Define parameters
#access_size = len(seq)
min_gc = 0
max_gc = 100
gc_ranges = 1
access_win_size = 100
seed_size = 6
seed_sizes = [seed_size * m for m in range(1, 4)]
accessibility_scores = []

# Loop through each row in the dataset
for i, row in tqdm(all_data_human_gene.iterrows(), total=len(all_data_human_gene)):
    seq = row[f'sense_with_flank_{FLANK_SIZE}nt']
    sense_start = row['sense_start']
    sense_length = row['sense_length']

    try:
        # Calculate accessibility DataFrame for the full sequence (with flanks)
        df_access = AccessCalculator.calc(
            seq, len(seq),
            min_gc, max_gc, gc_ranges,
            access_win_size, seed_sizes
        )

        # Compute where the sense region starts inside the flanked sequence
        flank_start = max(0, sense_start - FLANK_SIZE)
        sense_start_in_flank = sense_start - flank_start
        sense_end_in_flank = sense_start_in_flank + sense_length

        # Make sure the index range is within bounds
        if 0 <= sense_start_in_flank < len(df_access) and sense_end_in_flank <= len(df_access):
            mean_access = df_access['avg_access'].iloc[sense_start_in_flank:sense_end_in_flank].mean()
        else:
            mean_access = None

    except Exception as e:
        print(f"Error at row {i} | seq start: {sense_start} | error: {e}")
        mean_access = None

    accessibility_scores.append(mean_access)

# Add the computed accessibility values as a new column
all_data_human_gene['sense_avg_accessibility'] = accessibility_scores

ModuleNotFoundError: No module named 'rna_access'